In [1]:
import json
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import string

In [2]:
df = pd.read_json ('nfl6.json')

In [3]:
df.to_csv ('text.csv', index = None)

In [4]:
text = pd.read_csv("text.csv")

In [5]:
text

,main_category,question,nbestanswers,answer,id
0,News & Events,Why did the U.S Invade Iraq ?,"[""A small group of politicians believed strong...",A small group of politicians believed strongly...,2020338
1,Education & Reference,How to get rid of a beehive?,['Call an area apiarist. They should be able ...,Call an area apiarist. They should be able to...,2874684
2,Society & Culture,Why don't European restaurants serve water?,"[""There's a general belief in Europe (and in f...",There's a general belief in Europe (and in fac...,4193114
3,Cars & Transportation,Why hybrid cars gas mileage is better in city ?,"[""hybrid cars save energy in two ways: 1.by st...",hybrid cars save energy in two ways: 1.by stor...,1908421
4,Science & Mathematics,Can someone explain the theory of e=mc2?,['In general it means that in a very high spee...,In general it means that in a very high speed ...,3608897
...,...,...,...,...,...
87357,Education & Reference,"How did the phrase ""(someone) has gone missing...","['I think it\'s strange, too.. It sounds very ...","I think it's strange, too.. It sounds very col...",1134376
87358,Society & Culture,Why in the world do I have to press 1 to get E...,['Because there are some Spanish people. If it...,Because there are some Spanish people. If it a...,945182
87359,Science & Mathematics,Why chemicals should never be placed directly ...,"['Because chemicals react with eachother, it i...","Because chemicals react with eachother, it is ...",3433624
87360,Arts & Humanities,what do you consider..........?,"[""Never allow people to Ride on you.. Don't mi...",treat others as you want to be treated. becaus...,2900989


In [6]:
text['answer'][2]

"There's a general belief in Europe (and in fact elsewhere in the world, such as China) that it's unhealthy to drink water with a meal because it dilutes the digestive juices. Wine is OK because it stimulates digestion, and mineral waters are also considered digestives.. . In general, Europeans are much more concerned with the digestive process than Americans."

In [7]:
text['main_category'].unique()

array(['News & Events', 'Education & Reference', 'Society & Culture',
       'Cars & Transportation', 'Science & Mathematics',
       'Politics & Government', 'Pets', 'Food & Drink',
       'Business & Finance', 'Computers & Internet', 'Games & Recreation',
       'Health', 'Travel', 'Social Science', 'Yahoo! Products',
       'Consumer Electronics', 'Arts & Humanities', 'Dining Out',
       'Local Businesses', 'Asia Pacific', 'Yahoo!7 Products'],
      dtype=object)

In [8]:
text.drop(text[text['main_category'] == "Health"].index, inplace = True) 
text.drop(text[text['main_category'] == "News & events"].index, inplace = True) 
text.drop(text[text['main_category'] == "Cars & Transportation"].index, inplace = True) 

In [9]:
text.drop(text[text['main_category'] == "Asia Pacific"].index, inplace = True) 
text.drop(text[text['main_category'] == "Consumer Electronics"].index, inplace = True)
text.drop(text[text['main_category'] == "Local Businesses"].index, inplace = True)

In [10]:
text.drop(text[text['main_category'] == "Yahoo! Products"].index, inplace = True)
text.drop(text[text['main_category'] == "Dining Out"].index, inplace = True)
text.drop(text[text['main_category'] == "Games & Recreation"].index, inplace = True)

In [11]:
text.drop(text[text['main_category'] == "Yahoo!7 Products"].index, inplace = True) 
text.drop(text[text['main_category'] == "News & Events"].index, inplace = True) 
text.drop(text[text['main_category'] == "Business & Finance"].index, inplace = True)

In [12]:
text.drop(text[text['main_category'] == "Computers & Internet"].index, inplace = True)
text.drop(text[text['main_category'] == "Pets"].index, inplace = True) 
text.drop(text[text['main_category'] == "Politics & Government"].index, inplace = True)
text.drop(text[text['main_category'] == "Food & Drink"].index, inplace = True)
text.drop(text[text['main_category'] == "Travel"].index, inplace = True)
text.drop(text[text['main_category'] == "Science & Mathematics"].index, inplace = True)

In [13]:
text['main_category'].unique()

array(['Education & Reference', 'Society & Culture', 'Social Science',
       'Arts & Humanities'], dtype=object)

In [14]:
text['main_category'].describe()

count                 20922
unique                    4
top       Society & Culture
freq                   8651
Name: main_category, dtype: object

In [15]:
text = text.drop('id',axis=1)

In [16]:
text = text.drop('question',axis=1)

In [17]:
text

,main_category,nbestanswers,answer
1,Education & Reference,['Call an area apiarist. They should be able ...,Call an area apiarist. They should be able to...
2,Society & Culture,"[""There's a general belief in Europe (and in f...",There's a general belief in Europe (and in fac...
5,Education & Reference,"[""I used to work in a college admissions offic...",I used to work in a college admissions office ...
13,Society & Culture,['Possibly because African royalty used umbrel...,Possibly because African royalty used umbrella...
15,Society & Culture,"[""Eggnog is a popular wintertime drink. We do...",Eggnog is a popular wintertime drink. We do n...
...,...,...,...
87348,Society & Culture,['a friend is someone who can see your faults ...,a friend is someone who can see your faults an...
87355,Education & Reference,['Filipino physicist are physicists in the Phi...,Filipino physicist are physicists in the Phili...
87357,Education & Reference,"['I think it\'s strange, too.. It sounds very ...","I think it's strange, too.. It sounds very col..."
87358,Society & Culture,['Because there are some Spanish people. If it...,Because there are some Spanish people. If it a...


In [18]:
text = text.reset_index()

In [19]:
lemmer = WordNetLemmatizer()
corpus_ans = []
for i in range(0, len(text)):
    ans = re.sub(r'\[[0-9]*\]', ' ', text['answer'][i])
    ans = re.sub(r'[^a-zA-Z]', ' ', text['answer'][i])
    ans = re.sub(r'\n', '', text['answer'][i])
    ans = ans.lower()
    ans = ans.split()
    ans = [lemmer.lemmatize(word) for word in ans]
    ans = ' '.join(ans)
    corpus_ans.append(ans)

In [20]:
corpus_ans[0:5]

['call an area apiarist. they should be able to help you and would most likely remove them at no charge in exchange for the hive. the bee have value and they now belong to you.',
 "there's a general belief in europe (and in fact elsewhere in the world, such a china) that it's unhealthy to drink water with a meal because it dilutes the digestive juices. wine is ok because it stimulates digestion, and mineral water are also considered digestives.. . in general, european are much more concerned with the digestive process than americans.",
 "i used to work in a college admission office for a major university and they used a specific formula for calculating gpas. they did this to put everyone on an even playing field - so that high school that don't weight ap class differently would be measured the same a those that do. though i don't remember the specific calculation anymore, i definitely remember that ap class were given a higher score. i'd suggest you take the ap class and try for the be

In [21]:
text['answer'] = corpus_ans

In [22]:
text

,index,main_category,nbestanswers,answer
0,1,Education & Reference,['Call an area apiarist. They should be able ...,call an area apiarist. they should be able to ...
1,2,Society & Culture,"[""There's a general belief in Europe (and in f...",there's a general belief in europe (and in fac...
2,5,Education & Reference,"[""I used to work in a college admissions offic...",i used to work in a college admission office f...
3,13,Society & Culture,['Possibly because African royalty used umbrel...,possibly because african royalty used umbrella...
4,15,Society & Culture,"[""Eggnog is a popular wintertime drink. We do...",eggnog is a popular wintertime drink. we do no...
...,...,...,...,...
20917,87348,Society & Culture,['a friend is someone who can see your faults ...,a friend is someone who can see your fault and...
20918,87355,Education & Reference,['Filipino physicist are physicists in the Phi...,filipino physicist are physicist in the philip...
20919,87357,Education & Reference,"['I think it\'s strange, too.. It sounds very ...","i think it's strange, too.. it sound very coll..."
20920,87358,Society & Culture,['Because there are some Spanish people. If it...,because there are some spanish people. if it a...


In [23]:
text = text.drop('index',axis=1)
text = text.drop('nbestanswers',axis=1)
text = text.drop('main_category',axis=1)

In [24]:
text.to_csv("new2.csv",index=False)

In [25]:
df = pd.read_csv("new2.csv")

In [26]:
df

,answer
0,call an area apiarist. they should be able to ...
1,there's a general belief in europe (and in fac...
2,i used to work in a college admission office f...
3,possibly because african royalty used umbrella...
4,eggnog is a popular wintertime drink. we do no...
...,...
20917,a friend is someone who can see your fault and...
20918,filipino physicist are physicist in the philip...
20919,"i think it's strange, too.. it sound very coll..."
20920,because there are some spanish people. if it a...


In [27]:
f=open('final.txt','r',errors = 'ignore')

In [28]:
raw=f.read()

In [29]:
sent_tokens = nltk.sent_tokenize(raw)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
Greeting_Inputs = ("hello", "hi", "greetings", "sup", "good morning","hey","good evening","good afternoon")

In [32]:
Greeting_Responses = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [33]:
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in Greeting_Inputs:
            return random.choice(Greeting_Responses)

In [34]:
generic_response = ["My sole purpose is to talk. I am neither happy, sad or sleepy. What about you?", "Well,You already know the answer. Dont'you? "]

In [35]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [36]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [37]:
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [38]:
def response(user_response):
    bot_response=''
    sent_tokens.append(user_response)
    TfidfVector = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVector.fit_transform(sent_tokens)
    value = cosine_similarity(tfidf[-1], tfidf)
    final =value.argsort()[0][-2]
    flat = value.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        bot_response=bot_response+"Rephrase it into simpler words!"
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[final]
        return bot_response

In [43]:
flag=True
print("BOT: Let's talk! Keep the text short and use keywords. If you want to exit, type Bye or Thank you!")
while(flag==True):
    user_response = input('You : ')
    user_response=user_response.lower()
    keyword = "wisdom"
    keyword1 = "quote"
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("BOT : You are welcome!:)")
        else:
            if(greeting(user_response)!=None):
                print("BOT : "+greeting(user_response))
            elif(user_response.find("how are you") != -1 or user_response.find("do you feel") != -1 or user_response.find("do you have emotions")!=-1 or user_response.find("do you sleep")!=-1):
                print("BOT : "+random.choice(generic_response))
            elif(user_response.find("what is your name")!=-1):
                print("You can call me anything you want! I prefer Bot:)")
            elif(user_response.find(keyword)!=-1 or user_response.find(keyword1)!=-1):
                print("We've all got both light and dark inside us. What matters is the part we choose to act on. That's who we really are!")
            else:
                print("BOT: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("BOT: Bye! It was an interesting conversation.")

BOT: Let's talk! Keep the text short and use keywords. If you want to exit, type Bye or Thank you!
You : bye
BOT: Bye! It was an interesting conversation.
